## Bronze Layer

In [0]:

CREATE OR REFRESH STREAMING LIVE TABLE bronze_incident_response_seattle_911
COMMENT "These are the 911 incident response from Seattle"
AS
SELECT at_scene_time, cad_cdw_id, cad_event_number, census_tract, district_sector, event_clearance_code, event_clearance_date, event_clearance_description, event_clearance_group, event_clearance_subgroup, general_offense_number, hundred_block_location, incident_location, initial_type_description, initial_type_group, initial_type_subgroup, zone_beat
FROM cloud_files("${datasets.path}", "json", map("cloudFiles.inferColumnTypes", "true"))
GROUP BY at_scene_time, cad_cdw_id, cad_event_number, census_tract, district_sector, event_clearance_code, event_clearance_date, event_clearance_description, event_clearance_group, event_clearance_subgroup, general_offense_number, hundred_block_location, incident_location, initial_type_description, initial_type_group, initial_type_subgroup, zone_beat;


## Silver layer

In [0]:

CREATE OR REFRESH STREAMING LIVE TABLE silver_incident_response
(CONSTRAINT valid_record EXPECT (cad_cdw_id IS NOT NULL) ON VIOLATION DROP ROW)
COMMENT "These are the 911 incident response from everywhere"
AS
SELECT at_scene_time, cad_cdw_id, cad_event_number, census_tract, district_sector, event_clearance_code, event_clearance_date, event_clearance_description, event_clearance_group, event_clearance_subgroup, general_offense_number, hundred_block_location, 
CAST(SPLIT(SUBSTRING(incident_location, 2, LENGTH(incident_location) - 2), ', ')[0] AS INT) AS incident_latitude,
CAST(SPLIT(SUBSTRING(incident_location, 2, LENGTH(incident_location) - 2), ', ')[1] AS INT) AS incident_longitude, initial_type_description, initial_type_group, initial_type_subgroup, zone_beat
FROM STREAM(LIVE.bronze_incident_response_seattle_911);

## Gold Layer

In [0]:
CREATE OR REFRESH LIVE TABLE types_of_incidents_counts
COMMENT "This is a report of incidents by type"
AS 
SELECT initial_type_group, COUNT(*) AS num_of_incidents
FROM LIVE.silver_incident_response
GROUP BY initial_type_group;

